In [113]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from langdetect import detect
import re


In [114]:
inputDF = pd.read_csv('listings.csv.gz')

In [115]:
inputDF=inputDF[['id','name','summary','longitude','latitude','space','description','instant_bookable','neighborhood_overview','neighbourhood_cleansed','host_id','host_name','host_since',
                 'host_response_time','street', 'zipcode','review_scores_rating','property_type','room_type','accommodates','bathrooms','bedrooms','beds','reviews_per_month','amenities','cancellation_policy','number_of_reviews','price']]

In [116]:
inputDF.fillna(0, inplace=True)

In [117]:
#Creo la variabile prices in modo da pulire i prezzi e renderli poi un float
price = inputDF['price']
prices=[]
for p in price:
    p=float(p[1:].replace(',',''))
    prices.append(p)
#riaggiorno poi la colonna
inputDF['price']=prices

In [118]:
inputDF = inputDF[inputDF.price  > 0]

In [119]:
#creo una nuova variabile
summaryDF = inputDF[['summary','price']]
#tolgo gli elementi vuoti
summaryDF = summaryDF[pd.notnull(summaryDF['summary'])]
#tolgo gli elementi nulli
summaryDF = summaryDF[summaryDF['summary']!=0]
#metto in ordine di prezzo
summaryDF = summaryDF.sort_values('price',ascending=[0])
#prendo solo i primi 100
top100DF = summaryDF.head(100)
#da dataframe a list
a=top100DF.summary.values.tolist()
# detecto la lingua
c=[]
for i in range(0,len(a)):
    c.append(detect(a[i]))
#attacco l'info al dataframe
top100DF['lang']=c
#splitto i dataframe
top100DFen=top100DF[top100DF['lang'] == 'en']

<ipython-input-119-888d250b7d74>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100DF['lang']=c


In [120]:
import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
from nltk import tokenize

In [121]:
def text_emotion(df, column):
    '''
    Takes a DataFrame and a specified column of text and adds 10 columns to the
    DataFrame for each of the 10 emotions in the NRC Emotion Lexicon, with each
    column containing the value of the text in that emotions
    INPUT: DataFrame, string
    OUTPUT: the original DataFrame with ten new columns
    '''

    new_df = df.copy() 

    emolex_df = pd.read_csv('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt',
                            names=["word", "emotion", "association"],
                            sep='\t')
    emolex_df = pd.read_csv('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt',
                            names=["word", "emotion", "association"],
                            sep='\t')
    emolex_words = emolex_df.pivot(index='word',
                                   columns='emotion',
                                   values='association').reset_index()
    emotions = emolex_words.columns.drop('word')
    emo_df = pd.DataFrame(0, index=top100DF.index, columns=emotions)

    stemmer = SnowballStemmer("english")

    
    book = ''
    chapter = ''
    
    with tqdm(total=len(list(new_df.iterrows()))) as pbar:
        for i, row in new_df.iterrows():
            pbar.update(1)
            document = word_tokenize(new_df.loc[i]['summary'])
            for word in document:
                word = stemmer.stem(word.lower())
                emo_score = emolex_words[emolex_words.word == word]
                if not emo_score.empty:
                    for emotion in list(emotions):
                        emo_df.at[i, emotion] += emo_score[emotion]
            
    new_df = pd.concat([new_df, emo_df], axis=1)

    return new_df

In [122]:
hp_df = pd.DataFrame(data=top100DFen)

In [123]:
hp_df = text_emotion(hp_df, 'summary')

<ipython-input-121-1e66c539666c>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=len(list(new_df.iterrows()))) as pbar:


In [124]:
hp_df.head()

,summary,price,lang,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
60,You will be welcome in our cozy home. Our apar...,1200.0,en,0,1,0,0,2,1,6,0,0,3
305,"In the very heart of Bologna city centre, on t...",346.0,en,0,0,0,0,0,0,0,0,0,0
312,This comfortable two bedroom apartment (80 sq....,415.0,en,0,1,0,0,1,0,1,0,0,1
324,This comfortable three bedroom apartment (150 ...,623.0,en,0,1,0,0,1,0,1,0,0,1
325,This elegant apartment (170 sq.mt) is on the 2...,623.0,en,0,0,0,0,0,0,0,0,0,0


In [125]:
from nltk import word_tokenize
from nltk import tokenize
hp_df['summary'].dropna(inplace=True)
hp_df['word_count'] =hp_df['summary'].apply(tokenize.word_tokenize).apply(len)

In [126]:
emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']

In [127]:
for emotion in emotions:
    hp_df[emotion] = hp_df[emotion] / hp_df['word_count']

In [128]:
hp_df.head()

,summary,price,lang,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,word_count
60,You will be welcome in our cozy home. Our apar...,1200.0,en,0.0,0.020408,0.0,0.0,0.040816,0.020408,0.122449,0.0,0.0,0.061224,49.0
305,"In the very heart of Bologna city centre, on t...",346.0,en,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,43.0
312,This comfortable two bedroom apartment (80 sq....,415.0,en,0.0,0.022727,0.0,0.0,0.022727,0.000000,0.022727,0.0,0.0,0.022727,44.0
324,This comfortable three bedroom apartment (150 ...,623.0,en,0.0,0.020408,0.0,0.0,0.020408,0.000000,0.020408,0.0,0.0,0.020408,49.0
325,This elegant apartment (170 sq.mt) is on the 2...,623.0,en,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,47.0
